# Joining Tables with `dfply`

In [1]:
import pandas as pd
from dfply import *

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [2]:
dept = pd.read_csv("./data/department.csv")
dept

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [3]:
empl = pd.read_csv("./data/employee.csv")
empl

,LastName,DeptID
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,NaN


## Left Outer Join

Use `left_table >> dfply.left_join(right_table, by=common_col_name)`

In [4]:
empl >> left_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,NaN


## Rename when the column names are different

In [5]:
empl2 = empl.fillna(0) >> rename(deptIDL = 'DeptID') 
empl2

,LastName,deptIDL
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,0.0


In [6]:
empl2 >> left_join(dept, by='DeptID')

KeyError: 'DeptID'

In [7]:
empl2 >> rename(DeptID = 'deptIDL') >> left_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,0.0,NaN


## Right Outer Join

Use `right_table >> dfply.right_join(left_table, by=common_col_name)`

In [8]:
empl >> right_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,NaN,35.0,Marketing


## Full Outer Join

Use `left_table >> dfply.outer_join(right_table, by=common_col_name)`

In [9]:
empl >> outer_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,NaN
6,NaN,35.0,Marketing


## Inner Join

Use `left_table >> dfply.inner_join(right_table, by=common_col_name)`

In [10]:
empl >> inner_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical


## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [11]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/Teams.csv")

In [12]:
batting, teams = [pd.read_csv(f) for f in files]

## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [13]:
batting.head(2)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0


#### Selecting down to the necessary columns

In [14]:
batting_select = (batting
                  >> select(X.teamID,
                            X.yearID,
                            X.R)
                 )
batting_select.head()

,teamID,yearID,R
0,TRO,1871,0
1,RC1,1871,30
2,CL1,1871,28
3,WS3,1871,28
4,RC1,1871,29


In [15]:
batting_select.shape

(107429, 3)

## Things we need from `Teams.csv`

* Year
* Runs
* Team

In [16]:
teams.head(2)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1


#### Selecting down to the necessary columns

In [17]:
teams_select = (teams >> select(X.yearID,
                                X.teamID,
                                X.name,
                                X.park))
teams_select.head()

,yearID,teamID,name,park
0,1871,BS1,Boston Red Stockings,South End Grounds I
1,1871,CH1,Chicago White Stockings,Union Base-Ball Grounds
2,1871,CL1,Cleveland Forest Citys,National Association Grounds
3,1871,FW1,Fort Wayne Kekiongas,Hamilton Field
4,1871,NY2,New York Mutuals,Union Grounds (Brooklyn)


In [18]:
teams_select.shape

(2925, 4)

## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join? 
**Answer:** `batting >> left_join(teams)` on both year and 

## How to join on multiple columns

Pass a list of columns names to `by`

In [19]:
df_joined = batting_select >> left_join(teams_select, by=['teamID', 'yearID'])
df_joined.head()

,teamID,yearID,R,name,park
0,TRO,1871,0,Troy Haymakers,Haymakers' Grounds
1,RC1,1871,30,Rockford Forest Citys,Agricultural Society Fair Grounds
2,CL1,1871,28,Cleveland Forest Citys,National Association Grounds
3,WS3,1871,28,Washington Olympics,Olympics Grounds
4,RC1,1871,29,Rockford Forest Citys,Agricultural Society Fair Grounds


#### Filter, group by, and aggregate to get the desired result.

In [20]:
runs_per_team09 = (df_joined
                   >> filter_by(X.yearID == 2009)
                   >> group_by(X.name, X.park)
                   >> summarise(total_runs = X.R.sum()))
runs_per_team09.head()

,park,name,total_runs
0,Chase Field,Arizona Diamondbacks,720
1,Turner Field,Atlanta Braves,735
2,Oriole Park at Camden Yards,Baltimore Orioles,741
3,Fenway Park II,Boston Red Sox,872
4,Wrigley Field,Chicago Cubs,707


In [21]:
runs_per_team09.shape

(30, 3)

## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

#### Last example 

In [22]:
len(batting_select)*len(teams_select)

314229825

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [23]:
runs = (batting_select
        >> filter_by(X.yearID == 2009)
        >> group_by(X.teamID)
        >> summarise(total_runs = X.R.sum()))

runs.head()

,teamID,total_runs
0,ARI,720
1,ATL,735
2,BAL,741
3,BOS,872
4,CHA,724


#### Filter down to 2009 teams

In [24]:
teams09 = (teams_select
           >> filter_by(X.yearID == 2009)
           >> drop(X.yearID)
          )
teams09.head()

,teamID,name,park
2595,ARI,Arizona Diamondbacks,Chase Field
2596,ATL,Atlanta Braves,Turner Field
2597,BAL,Baltimore Orioles,Oriole Park at Camden Yards
2598,BOS,Boston Red Sox,Fenway Park II
2599,CHA,Chicago White Sox,U.S. Cellular Field


In [25]:
len(runs)*len(teams09)

900

In [26]:
runs_per_team09_new = (runs 
                       >> left_join(teams09, by='teamID')
                       >> drop(X.teamID))
runs_per_team09_new.head()

,total_runs,name,park
0,720,Arizona Diamondbacks,Chase Field
1,735,Atlanta Braves,Turner Field
2,741,Baltimore Orioles,Oriole Park at Camden Yards
3,872,Boston Red Sox,Fenway Park II
4,724,Chicago White Sox,U.S. Cellular Field


In [27]:
runs_per_team09_new.shape

(30, 3)

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

## <font color="red"> Exercise 1 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join methods presented above.

In [28]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/People.csv",
         "./data/baseball/core/Teams.csv")
batting, people, teams = [pd.read_csv(f) for f in files]

In [29]:
pd.set_option('display.max_columns', None)

In [30]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,0,0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,0,0,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,5,0,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,2,2,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,3,0,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [31]:
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,NaN,NaN,NaN,Don,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,NaN,NaN,NaN,Andy,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [32]:
teams.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,NaN,NaN,N,NaN,401,1372,426,70,37,3,60.0,19.0,73.0,16.0,NaN,NaN,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,NaN,NaN,N,NaN,302,1196,323,52,21,10,60.0,22.0,69.0,21.0,NaN,NaN,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,NaN,NaN,N,NaN,249,1186,328,35,40,7,26.0,25.0,18.0,8.0,NaN,NaN,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,NaN,NaN,N,NaN,137,746,178,19,8,2,33.0,9.0,16.0,4.0,NaN,NaN,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,NaN,NaN,N,NaN,302,1404,403,43,21,1,33.0,15.0,46.0,15.0,NaN,NaN,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2


In [33]:
# Preliminary step: what players have hit over 100 home runs?
(batting
    >> group_by(X.playerID, X.yearID)
    >> summarise(total_hr = X.HR.sum()) # in case stint matters
    >> ungroup
    >> filter_by(X.total_hr > 100)
    >> head
)

,yearID,playerID,total_hr


In [34]:
# How close did anyone get to 100?
# Note: not summing stints here, but as we see above, that doesn't really matter
# And I really doubt anyone appearing with multiple teams in a season is a high home run hitter anyway

(batting
    >> select(X.playerID, X.yearID, X.HR)
    >> arrange(X.HR, ascending=False)
    >> head(10)
)

,playerID,yearID,HR
80768,bondsba01,2001,73
77405,mcgwima01,1998,70
77752,sosasa01,1998,66
78721,mcgwima01,1999,65
81786,sosasa01,2001,64
79069,sosasa01,1999,63
41676,marisro01,1961,61
22271,ruthba01,1927,60
104124,stantmi03,2017,59
19054,ruthba01,1921,59


In [35]:
# So we weren't even close.
# Well, this is the process we would do if there were any results.
# Tested with threshold of 50 HR.
(batting
    >> select(X.playerID, X.yearID, X.teamID, X.HR)
    >> filter_by(X.HR > 100) # summing stints doesn't matter or we would do that before this part
    >> inner_join(teams, by=["teamID", "yearID"])
    >> inner_join(people, by="playerID")
    >> mutate(FullName = X.nameGiven + " " + X.nameLast) # I'd use an f-string, but those apparently don't work
    >> select(X.HR_x, X.yearID, X.FullName, X.name)
    >> rename(TeamName=X.name, Year=X.yearID, HomeRuns=X.HR_x)
    >> arrange(X.HomeRuns, X.Year, ascending=(False, True)) # sort by homeruns descending, then year ascending
)

,HomeRuns,Year,FullName,TeamName
